In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.1 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285398 sha256=0582addfa925fbf7bf5bd08a3b1bbcd8de4d92a218332699823c7b5206c080c6
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import random
import os
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS, ALSModel
import pyspark.sql.functions as F

In [3]:
RANDOM_SEED = 42
path = '/kaggle/input/rutube-data/'
save_path = '/kaggle/working/'

In [4]:
spark = SparkSession.builder \
    .master('local[*]') \
    .config("spark.driver.memory", "15g") \
    .appName('Recommender_system') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/10 03:24:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
interactions = spark.read.parquet(f"{path}player_starts_train.parquet").select('date', 'user_id', 'item_id', 'watch_time')
videos = spark.read.parquet(f"{path}videos.parquet").withColumn('duration', F.col('duration')/1000)
videos = videos.select('duration', 'item_id')


In [6]:
# interactions.show()

In [7]:
# videos.show()

In [8]:
join_interactions = interactions.join(videos,
                                      on='item_id',
                                      how='inner')
join_interactions = join_interactions.withColumn('norm_watch_time', F.col('watch_time')/F.col('duration'))
join_interactions = join_interactions.withColumn('item_id', F.regexp_replace("item_id", "video_", "").cast('integer'))
join_interactions = join_interactions.withColumn('user_id', F.regexp_replace("user_id", "user_", "").cast('integer'))
join_interactions = join_interactions.na.fill(value=0,subset=["norm_watch_time"]).select('norm_watch_time', 
                                                                                         'user_id', 
                                                                                         'item_id')

In [9]:
# train_data = join_interactions.filter(F.col('date') <= '2023-08-20')
# test_data = join_interactions.filter(F.col('date') > '2023-08-20')

In [10]:
# print("Number of rows in train_data:", train_data.count())
# print("Number of rows in test_data:", test_data.count())

In [ ]:
# Build the recommendation model using ALS on the training data
als = ALS(maxIter=15, regParam=0.01, userCol="user_id" ,itemCol="item_id", ratingCol="norm_watch_time")
#Fit the Model on Item-Based Data
model = als.fit(join_interactions)

23/09/10 03:29:30 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/09/10 03:30:13 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [ ]:
model.write().save('saved_model')

In [ ]:
model = ALSModel.load(f'{save_path}saved_model')


In [ ]:
users = spark.read.csv(f'{path}/sample_submission.csv', header=True).withColumn('user_id', 
                                                                        F.regexp_replace("user_id", 
                                                                                         "user_", "").cast('integer'))

In [ ]:
users.show()

In [ ]:
dfs_rec = model.recommendForUserSubset(users.select('user_id'), 10)

In [ ]:
dfs_rec.select('user_id', F.col('recommendations.item_id').alias('recs')).show()

In [ ]:
# df_final = dfs_rec.select('user_id', F.col('recommendations.item_id').alias('recs')).toPandas()
# df_final.to_csv('submission.csv', index=False)